In [80]:
import random
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [81]:
from Data.Dataloader import DataLoader

In [82]:
data = DataLoader()

================loading data================
原数据集训练样本数：19603，去除空值后样本数：10436
原数据集测试样本数：2198，去除空值后样本数：1168


In [83]:
X_train, y_train, X_test, y_test = data.getdata01()

In [84]:
#独热码归一化
y_train=np.argmax(np.array(y_train),axis=1)
y_test=np.argmax(np.array(y_test),axis=1)

In [85]:
x_test=X_test
y_test=y_test

In [86]:
x_train, x_val = X_train[:5226], X_train[5226:]
y_train, y_val = y_train[:5226], y_train[5226:]

In [87]:
# 
def make_pairs(x, y):
    """
      创建包含具有相应标签的心电图对的元组
      x : ECG List，这个列表中的每个索引对应一个心电图
      y : 标签列表，每个标签为一个整数值
      返回元组包含两个数组，（样例，标签）
      样例shape=(2*len(x),2,(特征))
      标签shape=()
    Returns:
        Tuple containing two numpy arrays as (pairs_of_samples, labels),
        where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
        labels are a binary array of shape (2len(x)).
    """
    num_classes = max(y) + 1
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]

    return np.array(pairs), np.array(labels).astype("float32")

In [88]:
# make train pairs
pairs_train, labels_train = make_pairs(x_train, y_train)
# make validation pairs
pairs_val, labels_val = make_pairs(x_val, y_val)
# make test pairs
pairs_test, labels_test = make_pairs(x_test, y_test)

In [89]:
x_train.shape,x_val.shape,y_train.shape,y_val.shape

((5226, 1000, 12), (5210, 1000, 12), (5226,), (5210,))

In [90]:
pairs_train.shape,labels_train.shape

((10452, 2, 1000, 12), (10452,))

In [91]:
pairs_test.shape,labels_test.shape

((2336, 2, 1000, 12), (2336,))

In [92]:
x_train_1 = pairs_train[:, 0]  # x_train_1.shape is (10452, 1000, 12)
x_train_2 = pairs_train[:, 1]

In [93]:
x_val_1 = pairs_val[:, 0]  # x_val_1.shape = (10452, 1000, 12)
x_val_2 = pairs_val[:, 1]

In [94]:
x_test_1 = pairs_test[:, 0]  # x_test_1.shape = (2338, 1000, 12)
x_test_2 = pairs_test[:, 1]

In [95]:
# 定义模型

#有两个输入层，每个输入层都通向自己的网络，该网络产生嵌入。
#Lambda层使用欧几里得距离将它们合并，并将合并的输出反馈到最终网络。


In [182]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """
    求两个向量的欧式距离
    参数：包含两个长度相同的张量
    返回：包含向量欧式距离的张量
        
    """
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

input = layers.Input((1000, 12))

x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv1D(32, 2, activation="tanh")(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(64, 2, activation="tanh")(x)
x = layers.MaxPooling1D(pool_size= 2)(x)
x = layers.Conv1D(32, 2, activation="tanh")(x)
x = layers.MaxPooling1D(pool_size= 2)(x)

x = layers.Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(64, activation="tanh")(x)
x = layers.Dense(2, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((1000, 12))
input_2 = layers.Input((1000, 12))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))



In [183]:

def loss(margin=1):
    """为'constrastive_loss'提供一个包含变量'margin'的封闭作用域。
    参数:margin:整数，定义了输入对之间距离的基线应归为异类。-(默认为1)。
    返回'constrastive_loss'函数，附带数据('margin')。
    """
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss


In [193]:
epochs = 10
batch_size = 64
margin = 0.7  # Margin for constrastive loss.

In [194]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
# siamese.summary()

In [195]:
history = siamese.fit(
    [x_train_1, x_train_2],
    labels_train,
    validation_data=([x_val_1, x_val_2], labels_val),
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/10
164/164 [==============================] - 15s 76ms/step - loss: 0.0048 - accuracy: 0.9841 - val_loss: 0.1836 - val_accuracy: 0.6150
Epoch 2/10
164/164 [==============================] - 10s 62ms/step - loss: 0.0046 - accuracy: 0.9865 - val_loss: 0.1573 - val_accuracy: 0.6830
Epoch 3/10
164/164 [==============================] - 10s 63ms/step - loss: 0.0049 - accuracy: 0.9866 - val_loss: 0.1679 - val_accuracy: 0.6544
Epoch 4/10
164/164 [==============================] - 10s 59ms/step - loss: 0.0046 - accuracy: 0.9869 - val_loss: 0.1584 - val_accuracy: 0.6801
Epoch 5/10
164/164 [==============================] - 7s 40ms/step - loss: 0.0041 - accuracy: 0.9887 - val_loss: 0.1645 - val_accuracy: 0.6811
Epoch 6/10
164/164 [==============================] - 7s 40ms/step - loss: 0.0045 - accuracy: 0.9886 - val_loss: 0.1578 - val_accuracy: 0.6843
Epoch 7/10
164/164 [==============================] - 7s 40ms/step - loss: 0.0047 - accuracy: 0.9875 - val_loss: 0.1645 - val_accuracy: 0.

In [ ]:

def plt_metric(history, metric, title, has_valid=True):

    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

In [ ]:
results = siamese.evaluate([x_test_1, x_test_2], labels_test)
print("test loss, test acc:", results)